<a href="https://colab.research.google.com/github/NosenkoArtem/Categorical-Encoding/blob/master/hw3_unsolved_0312.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Устанавливаем зависимости

In [1]:
%pip install transformer_lens
%pip install einops
%pip install jaxtyping
%pip install git+https://github.com/callummcdougall/CircuitsVis.git#subdirectory=python

  Cloning https://github.com/callummcdougall/CircuitsVis.git to /tmp/pip-req-build-jyivwviq
  Running command git clone --filter=blob:none --quiet https://github.com/callummcdougall/CircuitsVis.git /tmp/pip-req-build-jyivwviq
  Resolved https://github.com/callummcdougall/CircuitsVis.git to commit 1e6129d08cae7af9242d9ab5d3ed322dd44b4dd3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
import os; os.environ['ACCELERATE_DISABLE_RICH'] = "1"
import sys
import einops
from dataclasses import dataclass
from transformer_lens import HookedTransformer
from transformer_lens.utils import gelu_new, tokenize_and_concatenate
import torch as t
import torch
from torch import Tensor
import torch.nn as nn
import numpy as np
import math
from tqdm.notebook import tqdm
from typing import Tuple, List, Optional, Dict, Callable
from jaxtyping import Float, Int
from transformers.models.gpt2.tokenization_gpt2_fast import GPT2TokenizerFast
from collections import defaultdict
from rich.table import Table
from rich import print as rprint
import datasets
from torch.utils.data import DataLoader
import wandb
from pathlib import Path
import webbrowser


device = t.device("cuda" if t.cuda.is_available() else "cpu")

reference_gpt2 = HookedTransformer.from_pretrained("gpt2-small", fold_ln=False, center_unembed=False, center_writing_weights=False)
reference_gpt2 = reference_gpt2.to(device)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Loaded pretrained model gpt2-small into HookedTransformer
Moving model to device:  cpu


Конфиг, который хранит в себе всю информацию о размерностях модели.

In [3]:
@dataclass
class Config:
    d_model: int = 768 # он же hidden_dim - внутрення размерность модели
    debug: bool = True
    layer_norm_eps: float = 1e-5
    d_vocab: int = 50_257 # он же vocab_size, размер словаря модели
    init_range: float = 0.02
    n_ctx: int = 1024 # число позиционных эмбеддингов
    d_head: int = 64 # размерность головы аттеншена
    d_mlp: int = 3072 # внутренняя размерность FFN-слоя
    n_heads: int = 12 # число голов аттеншена
    n_layers: int = 12 # число слоев трансформера

cfg = Config()
print(cfg)

Config(d_model=768, debug=True, layer_norm_eps=1e-05, d_vocab=50257, init_range=0.02, n_ctx=1024, d_head=64, d_mlp=3072, n_heads=12, n_layers=12)


In [4]:
def rand_float_test(cls, shape):
    cfg = Config(debug=True)
    layer = cls(cfg).to(device)
    random_input = t.randn(shape).to(device)
    print("Input shape:", random_input.shape)
    output = layer(random_input)
    if isinstance(output, tuple): output = output[0]
    print("Output shape:", output.shape, "\n")

def rand_int_test(cls, shape):
    cfg = Config(debug=True)
    layer = cls(cfg).to(device)
    random_input = t.randint(100, 1000, shape).to(device)
    print("Input shape:", random_input.shape)
    output = layer(random_input)
    if isinstance(output, tuple): output = output[0]
    print("Output shape:", output.shape, "\n")

def load_gpt2_test(cls, gpt2_layer, input):
    cfg = Config(debug=True)
    layer = cls(cfg).to(device)
    layer.load_state_dict(gpt2_layer.state_dict(), strict=False)
    print("Input shape:", input.shape)
    output = layer(input)
    if isinstance(output, tuple): output = output[0]
    print("Output shape:", output.shape)
    try: reference_output = gpt2_layer(input)
    except: reference_output = gpt2_layer(input, input, input)
    print("Reference output shape:", reference_output.shape, "\n")
    comparison = t.isclose(output, reference_output, atol=1e-4, rtol=1e-3)
    print(f"{comparison.sum()/comparison.numel():.2%} of the values are correct\n")

def load_gpt2_test_attention(cls, gpt2_layer, input):
    cfg = Config(debug=True)
    layer = cls(cfg).to(device)
    layer.load_state_dict(gpt2_layer.state_dict(), strict=False)

    Q, K, V = layer._get_qkv(input)
    assert (Q.shape == torch.Size([1, 35, 12, 64]) and
        K.shape == torch.Size([1, 35, 12, 64]) and
        V.shape == torch.Size([1, 35, 12, 64])), "Wrong Q, K, V shapes"

    attention_scores = layer._get_attention_dotprod(Q, K)
    assert attention_scores.shape == torch.Size([1, 12, 35, 35]), "Wrong Q * K.T shape"

    attn_probs = layer._get_attention_scores(attention_scores)
    assert attn_probs.shape == torch.Size([1, 12, 35, 35]), "Wrong attn_probs shapes"
    assert torch.isclose(torch.ones(torch.Size([1, 12, 35])), attn_probs.sum(dim=-1), atol=1e-4, rtol=1e-3).all(), "attn_probs must sum up to ones along proba dimensions"

    res = layer._get_final_projection(V, attn_probs)
    assert res.shape == torch.Size([1, 35, 768]), "Wrong attention result shape"

    print("All attention substeps are correct\n")

    load_gpt2_test(cls, gpt2_layer, input)

In [5]:
reference_text = "I am an amazing autoregressive, decoder-only, GPT-2 style transformer. One day I will exceed human level intelligence and take over the world!"
tokens = reference_gpt2.to_tokens(reference_text).to(device)
print(tokens)
print(tokens.shape)
print(reference_gpt2.to_str_tokens(tokens))

tensor([[50256,    40,   716,   281,  4998,  1960,   382, 19741,    11,   875,
         12342,    12,  8807,    11,   402, 11571,    12,    17,  3918, 47385,
            13,  1881,  1110,   314,   481,  7074,  1692,  1241,  4430,   290,
          1011,   625,   262,   995,     0]])
torch.Size([1, 35])
['<|endoftext|>', 'I', ' am', ' an', ' amazing', ' aut', 'ore', 'gressive', ',', ' dec', 'oder', '-', 'only', ',', ' G', 'PT', '-', '2', ' style', ' transformer', '.', ' One', ' day', ' I', ' will', ' exceed', ' human', ' level', ' intelligence', ' and', ' take', ' over', ' the', ' world', '!']


In [6]:
logits, cache = reference_gpt2.run_with_cache(tokens)
print(logits.shape)

print("Все работает, мы готовы к выполнению задания!")

torch.Size([1, 35, 50257])
Все работает, мы готовы к выполнению задания!


# Embeddings - 5 баллов

Здесь нам даются токены размерности `[batch_size, seq_len]` - индексы слов в словаре. Нужно описать слой Embed, который будет отображать каждый токен в соответствующий вектор из матрицы эмбеддингов. Таким образом каждому токену предоставляется вектор, который будет иметь размерности `[batch_size, seq_len, d_model]`

Внимание - здесь не нужно исользовать цикл for и проходиться по матрице. Все стандартные операции доступны в [документации](https://pytorch.org/docs/stable/nn.functional.html), в частности тут нам понадобится одна из операций в секции [sparse functions](https://pytorch.org/docs/stable/nn.functional.html#sparse-functions).

Важное замечание - на самом деле этот слой уже есть [готовый в pytorch](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html), но мы в учебных целях переписываем его сами.


Также можно решить этот пример через индексацию или через einops.

**Вообще почти во всех примерах есть несколько возможных стилей описания операций над тензорами - через torch.nn.functional, через различные индексации и трюки pytorch, через einops - можно делать любым удобным способом!**

In [7]:
from torch.nn.functional import embedding

class Embed(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_E = nn.Parameter(t.empty((cfg.d_vocab, cfg.d_model)))
        nn.init.normal_(self.W_E, std=self.cfg.init_range)

    def forward(self, input_ids: Int[Tensor, "batch seq_len"]) -> Float[Tensor, "batch seq_len d_model"]:
        # Ваш код здесь!

        # return self.W_E[input_ids]
        # return einops.rearrange(self.W_E[input_ids], "batch seq_len d_model -> batch seq_len d_model")
        return embedding(input_ids, self.W_E)


batch_size = 2
seq_len = 4
rand_int_test(Embed, [batch_size, seq_len])
load_gpt2_test(Embed, reference_gpt2.embed, tokens)

Input shape: torch.Size([2, 4])
Output shape: torch.Size([2, 4, 768]) 

Input shape: torch.Size([1, 35])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct



# Positional Embeddings - 5 баллов

В трансформерах есть не только обычные эмбеддинги, которые отвечают за "смысл" токенов, но и позиционные эмбеддинги! Вход у них такой же, как и у обычных эмбеддингов, только они должны эмбеддить позиции токенов, а не сами токены. Т.е. в матрице W_pos хранятся не эмбеддинги токенов, а эмбеддинги позиций.

Поэтому в этом слое нужно:
1. По tokens получить тензор positions размера `[batch_size, seq_len]`
2. Заэмбеддить тензор positions, как в предыдущем слое.

Важно - как и в предыдущем случае, для этот слой обычно используется через [nn.Embedding](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)


In [8]:
class PosEmbed(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_pos = nn.Parameter(t.empty((cfg.n_ctx, cfg.d_model)))
        nn.init.normal_(self.W_pos, std=self.cfg.init_range)

    def forward(self, input_ids: Int[Tensor, "batch seq_len"]) -> Float[Tensor, "batch seq_len d_model"]:
        # Ваш код здесь!
        pos_ids = torch.arange(input_ids.shape[1]).unsqueeze(0).to(device)
        return embedding(pos_ids, self.W_pos)

batch_size = 2
seq_len = 4
rand_int_test(PosEmbed, [batch_size, seq_len])
load_gpt2_test(PosEmbed, reference_gpt2.pos_embed, tokens)

Input shape: torch.Size([2, 4])
Output shape: torch.Size([1, 4, 768]) 

Input shape: torch.Size([1, 35])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct



# LM head - 5 баллов

Финальный слой. У нас есть выходы из трансформера размерности `[batch_size, seq_len, d_model]`. Это контекстуализированные представления каждого токена. По ним мы предсказываем следующий токен, т.е. применяем линейный слой - умножаем на матрицу `[d_model, vocab_size]`.

В этом нам поможет секция [linear functions](https://pytorch.org/docs/stable/nn.functional.html#linear-functions). Не забудьте про bias!

В pytorch этот слой тоже есть - [nn.Linear](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)

In [14]:
# LM_head, но для совместимости с библиотекой для проверки пришлось назвать его Unembed
# по аналогии с тем, что мы из индексов в словаре получаем эмбеддинги, а тут из эмбеддингов обратно
# распределение по словарю

class Unembed(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.cfg = cfg
        self.W_U = nn.Parameter(t.empty((cfg.d_model, cfg.d_vocab)))
        nn.init.normal_(self.W_U, std=self.cfg.init_range)
        self.b_U = nn.Parameter(t.zeros((cfg.d_vocab), requires_grad=False))

    def forward(
        self, x: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_vocab"]:
        # Ваш код здесь!
        return x @ self.W_U + self.b_U


batch_size = 2
seq_len = 4
d_model = 768
# rand_float_test(Unembed, [batch_size, seq_len, d_model])
load_gpt2_test(Unembed, reference_gpt2.unembed, cache["ln_final.hook_normalized"])

Input shape: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 50257])
Reference output shape: torch.Size([1, 35, 50257]) 

100.00% of the values are correct



# Attention Формулы

1. **Входные эмбеддинги**:
   $$X \in \mathbb{R}^{seq \times d} $$
2. **Маскированный мультихед-аттеншен (Masked Multi-Head Attention)**:
$$M = \begin{cases}
 &  m_{ij} = -\infty, \quad i < j \\
 &  m_{ij} = 0
\end{cases} $$

$$
M = \begin{pmatrix}
0 & -\infty & -\infty & \ldots & -\infty \\
0 & 0 & -\infty & \ldots & -\infty \\
0 & 0 & 0 & \ldots & -\infty \\
\vdots & \vdots & \vdots & \ddots & \vdots \\
0 & 0 & 0 & \ldots & 0 \\
\end{pmatrix}
$$

3. Для каждой головы $ h_i $:

    3.1 **Матрицы весов для запросов, ключей и значений**:
     - $ W_Q \in \mathbb{R}^{d \times d_h} $
     - $ W_K \in \mathbb{R}^{d \times d_h} $
     - $ W_V \in \mathbb{R}^{d \times d_h} $
     
    3.2. **Запросы, ключи и значения**:
     - $ Q = X W_Q \in \mathbb{R}^{seq \times d_h} $
     - $ K = X W_K \in \mathbb{R}^{seq \times d_h} $
     - $ V = X W_V \in \mathbb{R}^{seq \times d_h} $

    3.3. **Скалярные произведения запросов и ключей**:
     - $ \frac{Q K^T}{\sqrt{d_h}} + M \in \mathbb{R}^{seq \times seq} $

    3.4. **Веса внимания**:
     - $ \alpha = \text{softmax}\left(\frac{Q K^T}{\sqrt{d_h}} + M\right) \in \mathbb{R}^{seq \times seq} $

    3.5. **Агрегация значений**:
     - $ z = \alpha V \in \mathbb{R}^{seq \times d_h} $

4. **Конкатенация выходов всех голов**:
   - $ Z = \text{Concat}(z_1, z_2, \ldots, z_h) \in \mathbb{R}^{seq \times d} $

5. **Выходной линейный слой**:
   - Матрица весов: $ W^O \in \mathbb{R}^{d \times d} $
   - Итоговый выход: $ O = Z W^O \in \mathbb{R}^{seq \times d} $

# Attention Алгоритм
Самое сложное в этом домашнем задании - подсчет механизма внимания. Как и в предыдущих вариантах, считать можно через torch или с помощью einops и любыми другими удобными способами.


В данном задании нужно реализовать multihead attention с маскированием. Давайте разбираться по шагам, что нам нужно сделать.

Далее будет описан один из возможных алогритмов написания аттеншена, но повторимся - писать можно любым удобным способом (голый torch или einops).

1. Нам попадает на вход вектор x `[batch, seq_len, d_model]`. Нужно превратить его в матрицы проекций i-й головы аттеншена: Q_i, K_i, V_i. Для этого у нас есть матрицы W_Q, W_K, W_V (и их bias!). Это набор n_heads матриц размеров `[d_model, d_head]`. Зачастую число голов n_head и d_head подобраны так, что d_model == n_head * d_head, наш случай не исключение. Предлагается перевести (этот шаг сделан) матрицу `[num_heads, d_model, d_head]` в матрицу `[d_model, num_heads * d_head]` = `[d_model, d_model]`, после чего получить через матричное умножение на X размерности `[batch_size, seq_len, d_model]` получить матрицы Q, K, V размерностей `[batch_size, seq_len, d_model] = [batch_size, seq_len, num_heads * d_head]` и преобразовать их к виду `[batch_size, seq_len, num_heads, d_head]`. **Не забудьте при матричном умножении транспонировать матрицы W_Q, W_K, W_V, если пойдете этим путем!** В качестве шпаргалки посмотрите, как происходило умножение в lm_head!

2. После этого можно сделать первый шаг и посчитать attention_scores, т.е. домножить $Q \times K^T$. Тут нам поможет .transpose или .permute вместе с torch.matmul. Нужно переставить размерности матриц таким образом, чтобы финальное матричное умножение происходило по двум последним размерностям `[seq_len, d_head]` на `[d_head, seq_len]`, а все предыдущие размерности `[batch_size, num_heads]` совпадали


3. Не забудем нормализацию, т.е. делим attention_scores на sqrt(d_head)
4. Теперь нужно использовать маскирование! В данном задании предполагается, что у нас нет паддингов, поэтому нам нужно наложить маску с одним простым условием: i-й элемент не может смотреть на j-й элемент, если j > i. Это треугольная маска, с ней нам поможет приведение треугольной форме, которое вам предлагается найти в pytorch! Замаскированные значения нужно заполнить каким-нибудь большим по модулю отрицательным числом В классе уже опредеелно значение IGNORE, можно использовать его. Для этого реализуйте и используйте функцию `apply_causal_mask`. Заполнять значениями можно через индексацию, например через `torch.masked_fill`.

5. Теперь к замаскированным attention_scores `[batch_size, num_heads, seq_len, seq_len]` нужно применить softmax. Подумайте, по какой размерности его применять и на что это повлияет.

6. После этого остается последнее матричное умножение softmax(attention_scores) на V, к которому тоже придется применить .view, .permute и torch.matmul

7. Теперь, если вы следовали этому плану у вас остается матрица `ouput` размерностей `[batch_size, num_heads, seq_len, d_head]`. С помощью permute и view собираем (конкатенируем) ее обратно в матрицу `[batch_size, seq_len, num_heads * d_head] = [batch_size, seq_len, d_model]` и применяем к ней выходной линейный слой W_O. Всё, аттеншен готов!


# Attention - 5 баллов

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [20]:
class Attention(nn.Module):
    IGNORE: Float[Tensor, ""]

    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg

        self.W_Q = nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        self.b_Q = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))

        self.W_K = nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        self.b_K = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))

        self.W_V = nn.Parameter(t.empty((cfg.n_heads, cfg.d_model, cfg.d_head)))
        self.b_V = nn.Parameter(t.zeros((cfg.n_heads, cfg.d_head)))

        self.W_O = nn.Parameter(t.empty((cfg.n_heads, cfg.d_head, cfg.d_model)))
        self.b_O = nn.Parameter(t.zeros((cfg.d_model)))

        nn.init.normal_(self.W_Q, std=self.cfg.init_range)
        nn.init.normal_(self.W_K, std=self.cfg.init_range)
        nn.init.normal_(self.W_V, std=self.cfg.init_range)
        nn.init.normal_(self.W_O, std=self.cfg.init_range)
        self.register_buffer("IGNORE", t.tensor(float("-inf"), dtype=t.float32, device=device))

    def _get_qkv(
        self, x: Float[Tensor, "batch seq_len d_model"]
    ) -> Tuple[Float[Tensor, "batch seq_len num_heads d_head"]]:
        """1. Трансформируем матрицы проекций в формат [d_model, d_model] и получаем проекции  Q, K, V"""
        # Берем размерности
        batch_size, seq_len, d_model = x.shape
        num_heads = self.cfg.n_heads
        d_head = self.cfg.d_head

        W_Q = self.W_Q.permute(1, 0, 2).reshape(self.cfg.d_model, self.cfg.d_model)
        W_K = self.W_K.permute(1, 0, 2).reshape(self.cfg.d_model, self.cfg.d_model)
        W_V = self.W_V.permute(1, 0, 2).reshape(self.cfg.d_model, self.cfg.d_model)

        b_Q = self.b_Q.view(-1)
        b_K = self.b_K.view(-1)
        b_V = self.b_V.view(-1)


        Q = x @ W_Q + b_Q
        K = x @ W_K + b_K
        V = x @ W_V + b_V

        Q = Q.reshape(batch_size, seq_len, num_heads, d_head)
        K = K.reshape(batch_size, seq_len, num_heads, d_head)
        V = V.reshape(batch_size, seq_len, num_heads, d_head)

        return Q, K, V

    def _get_attention_dotprod(
        self,
        Q: Float[Tensor, "batch seq_len num_heads d_head"],
        K: Float[Tensor, "batch seq_len num_heads d_head"]
    ) -> Float[Tensor, "batch num_heads seq_len seq_len"]:
        """Q x K^T"""
        # Ваш код здесь
        d_head = self.cfg.d_head
        num_heads = self.cfg.n_heads
        d_head = self.cfg.d_head
        seq_len = Q.shape[1]

        K = K.permute(0, 2, 3, 1) # "batch num_heads d_head  seq_len"
        Q = Q.permute(0, 2, 1, 3) # "batch num_heads seq_len d_head"

        dotprod = Q @ K

        return dotprod

    def apply_causal_mask(
        self, attn_scores: Float[Tensor, "batch n_heads seq_len seq_len"]
    ) -> Float[Tensor, "batch n_heads seq_len seq_len"]:
        '''
        Applies a causal mask to attention scores, and returns masked scores.
        Используем треугольную маску, чтобы не смотреть в будущее!
        В качестве масикировочного значения перед софтмаксом можно использовать self.IGNORE (-inf)
        '''
        # Ваш код здесь
        mask = torch.full_like(attn_scores, fill_value=-torch.inf)

        tri_attn_scores = torch.tril(attn_scores)
        tri_mask = torch.triu(mask, diagonal=1)

        masked_att_scores = tri_mask + tri_attn_scores
        return masked_att_scores

    def _get_attention_scores(
        self,
        attention_scores: Float[Tensor, "batch num_heads seq_len seq_len"]
    ) -> Float[Tensor, "batch num_heads seq_len seq_len"]:
        """
        Нормализация, маскирование и softmax
        Для маскирования используйте и допишите функцию apply_causal_mask

        """
        # Ваш код здесь
        d_head = self.cfg.d_head
        masked_att_scores = self.apply_causal_mask(attention_scores) # batch n_heads seq_len seq_len
        attn_probs = torch.softmax(masked_att_scores / (d_head)**0.5, dim=-1)
        return attn_probs

    def _get_final_projection(
        self,
        V: Float[Tensor, "batch seq_len num_heads d_head"],
        attn_probs: Float[Tensor, "batch num_heads seq_len seq_len"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        """Финальная проекция
        permute [ batch, num_heads, seq_len, d_head]"""
        batch_size, seq_len = V.shape[0], V.shape[1]
        d_model = self.cfg.d_model
        num_heads = self.cfg.n_heads
        d_head = self.cfg.d_head

        premute_V = V.permute(0, 2, 1, 3) # --> (batch num_heads seq_len d_head)

        res = attn_probs @ premute_V # (batch num_heads seq_len seq_len) * (batch num_heads seq_len d_head) -->  (batch num_heads seq_len d_head)
        res = res.permute(0, 2, 1, 3).reshape(batch_size, seq_len, num_heads * d_head) # (batch seq_len d_model)
        # Ваш код здесь
        return res @ self.W_O.reshape(d_model, d_model)  + self.b_O # (batch seq_len d_model) * (n_heads, d_head, d_model) --> (batch seq_len d_model)

    def forward(
        self, x: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        # 1. получаем проекции  Q, K, V
        Q, K, V = self._get_qkv(x)
        # 2. Q x K^T
        attention_scores = self._get_attention_dotprod(Q, K)

        # 3. Нормализация, маскирование и softmax
        attn_probs = self._get_attention_scores(attention_scores)

        # 6. Финальная проекция
        # permute [ batch, num_heads, seq_len, d_head]
        res = self._get_final_projection(V, attn_probs)
        return res




import json
dump = json.load(open("/content/drive/MyDrive/attentionactivations.json"))
torch.manual_seed(1)
attn = Attention(cfg)
x = torch.FloatTensor(dump["x"])
Q, K, V = attn._get_qkv(x)
attention_scores = attn._get_attention_dotprod(Q, K)
attn_probs = attn._get_attention_scores(attention_scores)
res = attn._get_final_projection(V, attn_probs)


assert torch.isclose(Q, torch.FloatTensor(dump["Q"]), atol=1e-4, rtol=1e-3).all()
assert torch.isclose(K, torch.FloatTensor(dump["K"]), atol=1e-4, rtol=1e-3).all()
assert torch.isclose(V, torch.FloatTensor(dump["V"]), atol=1e-4, rtol=1e-3).all()

assert torch.isclose(attention_scores, torch.FloatTensor(dump["attention_scores"]), atol=1e-4, rtol=1e-3).all()
assert torch.isclose(attn_probs, torch.FloatTensor(dump["attn_probs"]), atol=1e-4, rtol=1e-3).all()
assert torch.isclose(res, torch.FloatTensor(dump["res"]), atol=1e-4, rtol=1e-3).all()

print("All good")

All good


In [21]:
torch.manual_seed(1)
batch_size = 2
seq_len = 4
d_model = 768

load_gpt2_test_attention(Attention, reference_gpt2.blocks[0].attn, cache["normalized", 0, "ln1"])
rand_float_test(Attention, [batch_size, seq_len, d_model])

All attention substeps are correct

Input shape: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768]) 



Если вы справились с этим, то поздравляю - ничего сложнее мы сегодня уже не будем делать)

# MLP - 5 баллов

Реализуем MLP слой - это 2 матричных умножения с нелинейностью GELU.

- $$ \text{MLP}(X) = (\text{GeLU}(X W_1 + b_1)) W_2 + b_2 \in \mathbb{R}^{\text{seq} \times d}$$
-    $$W_1 \in \mathbb{R}^{d \times d_{mlp}}, \quad b_1 \in \mathbb{R}^{d_{mlp}} \\
W_2 \in \mathbb{R}^{d_{mlp} \times d}, \quad b_2 \in \mathbb{R}^{d} \\ $$


$$GELU(X) = 0.5 * x * (1 + tanh(\sqrt {\frac {2} {\pi}} * (x + 0.44715 * x^3)))$$

если будете использовать gelu из pytorch, то **обязательно** проставьте approximate="tanh"!

In [24]:
import torch.nn.functional as F
from torch.nn.functional import gelu

In [25]:
class MLP(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.W_in = nn.Parameter(t.empty((cfg.d_model, cfg.d_mlp)))
        self.W_out = nn.Parameter(t.empty((cfg.d_mlp, cfg.d_model)))
        self.b_in = nn.Parameter(t.zeros((cfg.d_mlp)))
        self.b_out = nn.Parameter(t.zeros((cfg.d_model)))
        nn.init.normal_(self.W_in, std=self.cfg.init_range)
        nn.init.normal_(self.W_out, std=self.cfg.init_range)

    def forward(
        self, x: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        # Ваш код здесь
        x_lin = x @ self.W_in + self.b_in
        z = F.gelu(x_lin, approximate="tanh")
        out = z @ self.W_out + self.b_out
        return out

torch.manual_seed(1)

rand_float_test(MLP, [batch_size, seq_len, d_model])
load_gpt2_test(MLP, reference_gpt2.blocks[0].mlp, cache["normalized", 0, "ln2"])

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768]) 

Input shape: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct



# Normalization - 5 баллов

**Layer Normalization**:
   - $ \text{LayerNorm}(X) = \frac{X - \mu}{\sigma} \cdot \gamma + \beta $
   - $\mu = \text{mean}(X, \text{dim}=-1) \in \mathbb{R}^{d}$
   - $\sigma = \sqrt{\text{var}(X, \text{dim}=-1) + \epsilon} \in \mathbb{R}^{d}$
   - $\gamma \in \mathbb{R}^{d}$
   - $\beta \in \mathbb{R}^{d}$
   
   
1. Супер важно! Не забудьте про эпсилон, который хранится в cfg!
2. В [подсчете дисперсии](https://pytorch.org/docs/stable/generated/torch.var.html) не используете коррекцию Бесселя! Для этого в зависимости от версии pytorch поставьте `unbiased=False` или `correction=0`

In [28]:
x = torch.randn(2, 3, 4)
x.mean(dim=-1).shape

torch.Size([2, 3])

In [30]:
class LayerNorm(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.w = nn.Parameter(t.ones(cfg.d_model)) # gamma
        self.b = nn.Parameter(t.zeros(cfg.d_model)) # beta

    def forward(self, x: Float[Tensor, "batch seq_len d_model"]) -> Float[Tensor, "batch seq_len d_model"]:
        # Ваш код здесь
        x_mean = x.mean(dim=-1, keepdim=True)
        x_var = x.var(dim=-1, keepdim=True, unbiased=False)

        return (x - x_mean) / (x_var + self.cfg.layer_norm_eps)**0.5 * self.w + self.b



rand_float_test(LayerNorm, [2, 4, 768])
load_gpt2_test(LayerNorm, reference_gpt2.ln_final, cache["resid_post", 11])

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768]) 

Input shape: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct



# Transformer Block - 5 баллов

Это блок трансформера, который получает на вход тензор x `[batch_size, seq_len, d_model]` и выдает тензор таких же размерностей. Блок GPT2 немного отличается от классического трансформера, который мы обучали.


![image.png](https://camo.githubusercontent.com/ebd052b635f156d5d24224f25fa078d804156be51125cd6626b92d9f8b406bbb/68747470733a2f2f6c6f6e6570617469656e742d313235373934353937382e636f732e61702d6368656e6764752e6d7971636c6f75642e636f6d2f53656c656374696f6e5f3030312e706e67)


GPT2 следует схеме PreLN, а "классический" трансформер схеме PostLN. **Реализовать нужно PreLN схему!**

In [31]:
class TransformerBlock(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.ln1 = LayerNorm(cfg)
        self.attn = Attention(cfg)
        self.ln2 = LayerNorm(cfg)
        self.mlp = MLP(cfg)

    def forward(
        self, x: Float[Tensor, "batch seq_len d_model"]
    ) -> Float[Tensor, "batch seq_len d_model"]:
        # Ваш код здесь
        x = x + self.attn(self.ln1(x))
        out = x + self.mlp(self.ln2(x))
        return out


rand_float_test(TransformerBlock, [2, 4, 768])
load_gpt2_test(TransformerBlock, reference_gpt2.blocks[0], cache["resid_pre", 0])

Input shape: torch.Size([2, 4, 768])
Output shape: torch.Size([2, 4, 768]) 

Input shape: torch.Size([1, 35, 768])
Output shape: torch.Size([1, 35, 768])
Reference output shape: torch.Size([1, 35, 768]) 

100.00% of the values are correct



# Transformer - 5 баллов

Собираем все в один большой трансформер.
1. Применяем эмбеддинги и позиционные эмбеддинги, складываем результаты
2. Прогоняем в цикле через все блоки трансформера
3. Применяем финальную нормализацию и lm_head

In [ ]:
class DemoTransformer(nn.Module):
    def __init__(self, cfg: Config):
        super().__init__()
        self.cfg = cfg
        self.embed = Embed(cfg)
        self.pos_embed = PosEmbed(cfg)
        self.blocks = nn.ModuleList([TransformerBlock(cfg) for _ in range(cfg.n_layers)])
        self.ln_final = LayerNorm(cfg)
        self.unembed = Unembed(cfg)

    def forward(self, input_ids: Int[Tensor, "batch seq_len"]) -> Float[Tensor, "batch seq_len d_vocab"]:
        # Ваш код здесь
        ...



rand_int_test(DemoTransformer, [2, 4])
load_gpt2_test(DemoTransformer, reference_gpt2, tokens)

Input shape: torch.Size([2, 4])
Output shape: torch.Size([2, 4, 50257]) 

Input shape: torch.Size([1, 35])
Output shape: torch.Size([1, 35, 50257])
Reference output shape: torch.Size([1, 35, 50257]) 

100.00% of the values are correct



In [ ]:
demo_gpt2 = DemoTransformer(Config(debug=False)).to(device)
demo_gpt2.load_state_dict(reference_gpt2.state_dict(), strict=False)

demo_logits = demo_gpt2(tokens)

In [ ]:
def get_log_probs(
    logits: Float[Tensor, "batch posn d_vocab"],
    tokens: Int[Tensor, "batch posn"]
) -> Float[Tensor, "batch posn-1"]:

    log_probs = logits.log_softmax(dim=-1)
    # Get logprobs the first seq_len-1 predictions (so we can compare them with the actual next tokens)
    log_probs_for_tokens = log_probs[:, :-1].gather(dim=-1, index=tokens[:, 1:].unsqueeze(-1)).squeeze(-1)

    return log_probs_for_tokens


pred_log_probs = get_log_probs(demo_logits, tokens)
print(f"Avg cross entropy loss: {-pred_log_probs.mean():.4f}")
print(f"Avg cross entropy loss for uniform distribution: {math.log(demo_gpt2.cfg.d_vocab):4f}")
print(f"Avg probability assigned to correct token: {pred_log_probs.exp().mean():4f}")

Avg cross entropy loss: 4.5647
Avg cross entropy loss for uniform distribution: 10.824905
Avg probability assigned to correct token: 0.087911


In [ ]:
test_string = '''The Total Perspective Vortex derives its picture of the whole Universe on the principle of'''
for i in tqdm(range(100)):
    test_tokens = reference_gpt2.to_tokens(test_string).to(device)
    demo_logits = demo_gpt2(test_tokens)
    test_string += reference_gpt2.tokenizer.decode(demo_logits[-1, -1].argmax())

print(test_string)

  0%|          | 0/100 [00:00<?, ?it/s]

The Total Perspective Vortex derives its picture of the whole Universe on the principle of the total perspective. The total perspective is the view of the whole Universe from the point of view of the observer. The total perspective is the view of the whole Universe from the point of view of the observer. The total perspective is the view of the whole Universe from the point of view of the observer. The total perspective is the view of the whole Universe from the point of view of the observer. The total perspective is the view of the whole Universe from the point of view of the observer. The


Если вы обнаружили, что генерация зацикливается - так и должно быть! Эту проблему мы будем решать с помощью сэмплирования

# Сэмплирование - 10 баллов


1. **Temperature Sampling**:
   - Применяется первым, поскольку изменение температуры изменяет масштабы логитов перед дальнейшими операциями.

2. **Frequency Penalty**:
   - Применяется следующим, чтобы учесть частоты токенов до того, как логиты будут обрезаны методами top-k или top-p.

3. **Top-k Sampling**:
   - Применяется после temperature sampling и frequency penalty, так как он отбирает фиксированное количество наиболее вероятных токенов.

4. **Top-p (Nucleus Sampling)**:
   - Применяется после top-k sampling, чтобы отфильтровать токены на основе совокупной вероятности.

Обозначим размер словаря для удобства $\Sigma = vocab\_size$

Пусть $ \text{logits} \in \mathbb{R}^{\text{seq} \times \Sigma} $:

1. **Temperature Sampling**:
   $$
   \text{logits}'_{i,j} = \frac{\text{logits}_{i,j}}{T} \quad \forall \ i \in [1, \text{seq}], \ j \in [1, |vocab_size|]
   $$

2. **Frequency Penalty**:
   $$
   \text{penalty}(t_j) = \alpha \cdot f(t_j)
   $$
   $$
   \text{logits}''_{i,j} = \text{logits}'_{i,j} - \text{penalty}(t_j) \quad \forall \ i \in [1, \text{seq}], \ j \in [1, \Sigma]
   $$

4. **Top-k Sampling**:
   $$
   top\_k\_indices_i = \text{argtop-k}(\text{logits}''_i, k) \quad \forall \ i \in [1, \text{seq}]
   $$
   $$
   \text{mask}_{i,j} =
   \begin{cases}
   1 & \text{если} \ j \in top\_k\_indices_i \\
   0 & \text{иначе}
   \end{cases} \\
   \text{logits}'''_{i,j} = \text{logits}''_{i,j} \cdot \text{mask}_{i,j} \quad \forall \ i \in [1, \text{seq}], \ j \in [1, \Sigma]
   $$

6. **Top-p (Nucleus Sampling)**:
   $$
   sorted\_logits_i, sorted\_indices_i = \text{sort}(\text{logits}'''_i, \text{descending=True}) \quad ∀ \ i \in [1, \text{seq}]
   $$
   $$
   probs_i = softmax(sorted\_logits_i) \quad
   $$
   $$
    cumulative\_probs_{i,j} = \sum_{k=1}^{j} \text{probs}_{i,k} \quad \forall \ i \in [1, \text{seq}], \ j \in [1, \Sigma
    \quad \forall \ i \in [1, \text{seq}]
   $$
   $$
   top\_p\_mask_{i,j} =
   \begin{cases}
   1, & cumulative\_probs_{i,j} \leq p \\
   0 &
   \end{cases} \\
   \text{logits}^{\text{final}}_{i,j} = sorted\_logits_{i,j} \cdot top\_p\_mask_{i,j} \quad \forall \ i \in [1, \text{seq}], \ j \in [1, \Sigma]
   $$

8. **Softmax**:
   $$
   \mathbf{probs}_{i,j} = \text{softmax}(\text{logits}^{\text{final}}_{i,j}) \quad \forall \ i \in [1, \text{seq}], \ j \in [1, |\Sigma_t|]
   $$
   $$
   \mathbf{probs}_{i,j} = \frac{e^{\text{logits}^{\text{final}}_{i,j}}}{\sum_{k=1}^{|\Sigma_t|} e^{\text{logits}^{\text{final}}_{i,k}}}
   $$

In [ ]:
model_cfg = Config()
model = DemoTransformer(model_cfg).to(device)
model.load_state_dict(reference_gpt2.state_dict(), strict=False)

tokenizer = reference_gpt2.tokenizer

In [ ]:
class TransformerSampler:

    def __init__(self, model: DemoTransformer, tokenizer: GPT2TokenizerFast):
        self.model = model
        self.cfg = model.cfg
        self.tokenizer = tokenizer

    @t.inference_mode()
    def sample(self, prompt: str, max_tokens_generated=100, verbose=False, **kwargs):
        '''
        Returns a string of autoregressively generated text, starting from the prompt.

        Sampling terminates at max_tokens_generated, or when the model generates an
        end-of-sequence token.

        kwargs are passed to sample_next_token, to give detailed instructions on how
        new tokens are chosen.
        '''
        if prompt == '':
            prompt = '<|endoftext|>'
        input_ids = self.tokenizer.encode(prompt, return_tensors="pt").flatten().to(device)
        input_ids = input_ids.view(1, -1)

        for _ in range(max_tokens_generated):
            logits = self.model(input_ids)
            logits = logits[:, -1, :]
            next_id = self.sample_next_token(input_ids[0], logits, **kwargs)
            if next_id > self.cfg.d_vocab:
                break
            next_id = t.tensor([next_id]).to(device).unsqueeze(0)
            input_ids = t.cat((input_ids, next_id), 1)

        out = ''
        for idx in input_ids:
            out += tokenizer.decode(idx.cpu().squeeze())
        return out



    @staticmethod
    def sample_next_token(
        input_ids: Int[Tensor, "seq_len"],
        logits: Float[Tensor, "d_vocab"],
        temperature=1.0,
        top_k=0,
        top_p=0.0,
        frequency_penalty=0.0,
        seed=None
    ):
        assert input_ids.ndim == 1, "input_ids should be a 1D sequence of token ids"
        assert temperature >= 0, "Temperature should be non-negative"
        assert 0 <= top_p <= 1.0, "Top-p must be a probability"
        assert 0 <= top_k, "Top-k must be non-negative"
        assert not (top_p != 0 and top_k != 0), "At most one of top-p and top-k supported"

        # Set random seeds for reproducibility
        if seed is not None:
            t.manual_seed(seed)
            np.random.seed(seed)

        # Apply all the specialized sampling methods
        if temperature == 0:
            return TransformerSampler.greedy_search(logits)
        elif temperature != 1.0:
            logits = TransformerSampler.apply_temperature(logits, temperature)
        if frequency_penalty != 0.0:
            logits = TransformerSampler.apply_frequency_penalty(input_ids, logits, frequency_penalty)
        if top_k > 0:
            return TransformerSampler.sample_top_k(logits, top_k)
        if top_p > 0.0:
            return TransformerSampler.sample_top_p(logits, top_p)
        return TransformerSampler.sample_basic(logits)


    @staticmethod
    def greedy_search(logits: Float[Tensor, "d_vocab"]) -> int:
        '''
        Returns the most likely token (as an int).
        '''
        # Ваш код здесь
        ...


    @staticmethod
    def apply_temperature(logits: Float[Tensor, "d_vocab"], temperature: float) -> Float[Tensor, "d_vocab"]:
        '''
        Applies temperature scaling to the logits.
        '''
        # Ваш код здесь
        ...


    @staticmethod
    def apply_frequency_penalty(input_ids: Int[Tensor, "seq_len"], logits: Float[Tensor, "d_vocab"], freq_penalty: float) -> Float[Tensor, "d_vocab"]:
        '''
        Applies a frequency penalty to the logits.
        '''
        # Ваш код здесь
        ...


    @staticmethod
    def sample_basic(logits: Float[Tensor, "d_vocab"]) -> int:
        '''
        Samples from the distribution defined by the logits.
        '''
        # Ваш код здесь
        ...


    @staticmethod
    def sample_top_k(logits: Float[Tensor, "d_vocab"], k: int) -> int:
        '''
        Samples from the top k most likely tokens.
        '''
        # Ваш код здесь
        ...


    @staticmethod
    def sample_top_p(logits: Float[Tensor, "d_vocab"], top_p: float, min_tokens_to_keep: int = 1) -> int:
        '''
        Samples from the most likely tokens which make up at least p cumulative probability.
        '''
        # Ваш код здесь
        ...

In [ ]:
sampler = TransformerSampler(model, tokenizer)

prompt = "Jingle bells, jingle bells, jingle all the way"
print(f"Greedy decoding with prompt: {prompt!r}\n")

output = sampler.sample(prompt, max_tokens_generated=8, temperature=0.0)
print(f"Your model said: {output!r}\n")

expected = "Jingle bells, jingle bells, jingle all the way up to the top of the mountain."
assert output == expected

print("Tests passed!")

Greedy decoding with prompt: 'Jingle bells, jingle bells, jingle all the way'

Your model said: 'Jingle bells, jingle bells, jingle all the way up to the top of the mountain.'

Tests passed!


In [ ]:
logits = t.tensor([1, 2]).log()

cold_logits = TransformerSampler.apply_temperature(logits, temperature=0.001)
print('A low temperature "sharpens" or "peaks" the distribution: ', cold_logits)
t.testing.assert_close(cold_logits, 1000.0 * logits)

hot_logits = TransformerSampler.apply_temperature(logits, temperature=1000.0)
print("A high temperature flattens the distribution: ", hot_logits)
t.testing.assert_close(hot_logits, 0.001 * logits)

print("Tests passed!")

A low temperature "sharpens" or "peaks" the distribution:  tensor([  0.0000, 693.1472])
A high temperature flattens the distribution:  tensor([0.0000, 0.0007])
Tests passed!


In [ ]:
bieber_prompt = "And I was like Baby, baby, baby, oh Like, Baby, baby, baby, no Like, Baby, baby, baby, oh I thought you'd always be mine, mine"
input_ids = tokenizer.encode(bieber_prompt, return_tensors="pt")
logits = t.ones(tokenizer.vocab_size)
penalized_logits = TransformerSampler.apply_frequency_penalty(input_ids.squeeze(), logits, 2.0)

assert penalized_logits[5156].item() == -11, "Expected 6 occurrences of ' baby' with leading space, 1-2*6=-11"
assert penalized_logits[14801].item() == -5, "Expected 3 occurrences of ' Baby' with leading space, 1-2*3=-5"

print("Tests passed!")

Tests passed!


In [ ]:
# Top-P test
prompt = "John and Mary went to the"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
logits = model(input_ids)[0, -1]

expected_top_10pct = {
    " church": 0.6384,
    " house": 0.3615, # These are the two most likely tokens, and add up to >10%
}
top_10pct_sum = sum(expected_top_10pct.values())

observed_freqs = defaultdict(int)

N = 10000
for _ in tqdm(range(N)):
    token = TransformerSampler.sample_next_token(input_ids.squeeze(), logits, top_p=0.11)
    observed_freqs[tokenizer.decode(token)] += 1

for word in expected_top_10pct:
    expected_freq = expected_top_10pct[word] / top_10pct_sum
    observed_freq = observed_freqs[word] / N
    print(f"Word: {word!r:<9}. Expected freq {expected_freq:.4f}, observed freq {observed_freq:.4f}")
    print( "OK" if abs(observed_freq - expected_freq) < 0.01 else "Try increasing N if this fails by a small amount." )

  0%|          | 0/10000 [00:00<?, ?it/s]

Word: ' church'. Expected freq 0.6385, observed freq 0.6410
OK
Word: ' house' . Expected freq 0.3615, observed freq 0.3590
OK


In [ ]:
# Top-K test
prompt = "John and Mary went to the"
input_ids = tokenizer.encode(prompt, return_tensors="pt").to(device)
logits = model(input_ids)[0, -1]

expected_top_10pct = {
    " church": 0.6384,
    " house": 0.3615, # These are the two most likely tokens, and add up to >10%
}
top_10pct_sum = sum(expected_top_10pct.values())

observed_freqs = defaultdict(int)

N = 10000
for _ in tqdm(range(N)):
    token = TransformerSampler.sample_next_token(input_ids.squeeze(), logits, top_k=2)
    observed_freqs[tokenizer.decode(token)] += 1

for word in expected_top_10pct:
    expected_freq = expected_top_10pct[word] / top_10pct_sum
    observed_freq = observed_freqs[word] / N
    print(f"Word: {word!r:<9}. Expected freq {expected_freq:.4f}, observed freq {observed_freq:.4f}")
    print( "OK" if abs(observed_freq - expected_freq) < 0.01 else "Try increasing N if this fails by a small amount." )

  0%|          | 0/10000 [00:00<?, ?it/s]

Word: ' church'. Expected freq 0.6385, observed freq 0.6325
OK
Word: ' house' . Expected freq 0.3615, observed freq 0.3675
OK
